In [9]:
# Importing necessary libraries
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error


# 1. Data Loading and Preprocessing


In [10]:
merged_data = pd.read_csv('/Users/eduardoangeli/Library/CloudStorage/OneDrive-Fanshawec.ca/Capstone/Dataset/merged_data.csv')
merged_data.head()

,home_ownership,annual_inc,loan_amnt,dti,emp_length,label
0,MORTGAGE,55000.0,3600.0,0.1998,10+ years,1
1,MORTGAGE,65000.0,24700.0,0.1998,10+ years,1
2,MORTGAGE,63000.0,20000.0,0.1998,10+ years,1
3,MORTGAGE,110000.0,35000.0,0.1998,10+ years,1
4,MORTGAGE,104433.0,10400.0,0.1998,3 years,1


# 2. Feature Engineering

In [11]:
# One-hot encoding categorical columns ( encoding to categorical variables)
encoded_data = pd.get_dummies(merged_data, columns=['home_ownership', 'emp_length'])

# Splitting data and scaling features (split our data into training and test sets and scale, to normalizing data)
X = encoded_data.drop(columns=['label']) 
y = merged_data['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


# Handle NaN values
if np.isnan(X_train_scaled).any():
    nan_columns = np.where(np.isnan(X_train_scaled).any(axis=0))[0]
    for col in nan_columns:
        col_mean = np.nanmean(X_train_scaled[:, col])
        X_train_scaled[np.isnan(X_train_scaled[:, col]), col] = col_mean
        X_test_scaled[np.isnan(X_test_scaled[:, col]), col] = col_mean

# 3. Model Training and Evaluation

In [12]:
# Train logistic regression model
classifier = LogisticRegression(max_iter=1000, random_state=42)
classifier.fit(X_train_scaled, y_train)

# Evaluate model
y_pred = classifier.predict(X_test_scaled)
mse = mean_squared_error(y_test, y_pred)

# 4. Return value to the user

In [13]:
def generate_random_user_predict(classifier, X_train, scaler):
    """Generate a random example of a new user and predict loan acceptance."""
    
    # Randomly select values for each feature
    home_ownership = np.random.choice(['MORTGAGE', 'RENT', 'OWN'])
    annual_inc = np.random.uniform(20000, 150000)  # Random income between 20k and 150k
    loan_amnt = np.random.uniform(500, 40000)  # Random loan amount between $500 and $40k
    dti = np.random.uniform(0.1, 0.9)  # Random dti between 10% and 90%
    emp_length = np.random.choice(['< 1 year', '1 year', '2 years', '3 years', '4 years', '5 years',
                                   '6 years', '7 years', '8 years', '9 years', '10+ years'])
    
    # Creating a DataFrame for the user
    user_data = pd.DataFrame({
        'home_ownership': [home_ownership],
        'annual_inc': [annual_inc],
        'loan_amnt': [loan_amnt],
        'dti': [dti],
        'emp_length': [emp_length]
    })
    
    # One-hot encoding
    user_encoded = pd.get_dummies(user_data, columns=['home_ownership', 'emp_length'])
    
    # Ensuring the user_encoded has all the columns that the model expects
    # If not, they're filled with zeros (because of one-hot encoding)
    missing_cols = set(X_train.columns) - set(user_encoded.columns)
    for col in missing_cols:
        user_encoded[col] = 0
    user_encoded = user_encoded[X_train.columns]
    
    # Scaling the user data
    user_scaled = scaler.transform(user_encoded)
    
    # Making prediction
    prediction = classifier.predict(user_scaled)
    
    # Printing/Returning the result
    if prediction[0] == 1:
        print(f"""Generated user with:
Home Ownership: {home_ownership}
Annual Income: ${annual_inc:.2f}
Loan Amount: ${loan_amnt:.2f}
DTI: {dti*100:.2f}%
Employment Length: {emp_length}

Prediction: Loan ACCEPTED""")
    else:
        print(f"""Generated user with:
Home Ownership: {home_ownership}
Annual Income: ${annual_inc:.2f}
Loan Amount: ${loan_amnt:.2f}
DTI: {dti*100:.2f}%
Employment Length: {emp_length}

Prediction: Loan REJECTED""")

# Calling the function
generate_random_user_predict(classifier, X_train, scaler)

Generated user with:
Home Ownership: RENT
Annual Income: $83962.57
Loan Amount: $8912.23
DTI: 74.16%
Employment Length: 9 years

Prediction: Loan ACCEPTED


# Convert to pickle file to deploy

In [14]:
with open('model.pkl', 'wb') as file:
    pickle.dump(classifier, file)